In [3]:
import cv2
import numpy as np
from skimage import measure
from skimage.measure import regionprops

def detect_text_regions(image):
    """Detect text regions using morphology"""
    # Apply adaptive thresholding for better text detection
    thresh = cv2.adaptiveThreshold(image, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV, 11, 2)
    
    # Use morphological operations to enhance text regions
    kernel = np.ones((3, 3), np.uint8)
    dilated = cv2.dilate(thresh, kernel, iterations=1)
    
    # Find contours of potential text regions
    contours, _ = cv2.findContours(dilated, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Create a mask for text regions
    text_mask = np.zeros_like(image)
    
    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)
        aspect_ratio = w / float(h)
        area = w * h
        density = cv2.countNonZero(thresh[y:y+h, x:x+w]) / area
        
        # Characteristics of typical text regions
        if (0.1 < aspect_ratio < 15 and 
            10 < w < 300 and 
            5 < h < 100 and 
            0.1 < density < 0.9):
            # Mark as text region
            cv2.rectangle(text_mask, (x, y), (x+w, y+h), 255, -1)
    
    return text_mask

def detect_signatures(image, text_mask):
    """Detect signatures by excluding text regions"""
    # Remove text regions from the image
    image_no_text = image.copy()
    image_no_text[text_mask > 0] = 255
    
    # Threshold the image to isolate dark regions (potential signatures)
    _, binary = cv2.threshold(image_no_text, 200, 255, cv2.THRESH_BINARY_INV)
    
    # Apply morphological operations to enhance signature-like regions
    kernel = np.ones((2, 2), np.uint8)
    binary = cv2.dilate(binary, kernel, iterations=1)
    binary = cv2.erode(binary, kernel, iterations=1)
    
    # Connected component analysis
    blobs_labels = measure.label(binary, connectivity=2)
    regions = regionprops(blobs_labels)
    
    # Filter regions based on signature characteristics
    signature_mask = np.zeros_like(image)
    
    for region in regions:
        if region.area < 100:  # Skip very small regions
            continue
            
        y0, x0, y1, x1 = region.bbox
        w, h = x1 - x0, y1 - y0
        
        aspect_ratio = w / float(h)
        density = region.area / (w * h)
        perimeter_area_ratio = region.perimeter / region.area if region.area > 0 else 0
        
        # Signature characteristics: irregular shape, sparse density
        if (0.5 < aspect_ratio < 10 and 
            0.05 < density < 0.2 and 
            min(w, h) > 20 and 
            max(w, h) > 50 and
            region.solidity < 0.6 and  # Less solid, more irregular
            perimeter_area_ratio > 0.4):  # More complex boundary
            # Set only the exact region pixels
            signature_mask[blobs_labels == region.label] = 255
    
    return signature_mask

def detect_stamps(image):
    """Detect stamps using connected component analysis"""
    # Blur to reduce noise
    blurred = cv2.GaussianBlur(image.copy(), (5, 5), 0)
    
    # Adaptive thresholding to highlight dark stamp regions
    thresh = cv2.adaptiveThreshold(blurred, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY_INV, 11, 2)
    
    # Connected component analysis
    blobs_labels = measure.label(thresh, connectivity=2)
    regions = regionprops(blobs_labels)
    
    stamp_mask = np.zeros_like(image)
    
    for region in regions:
        if region.area < 1000 or region.area > 50000:  # Size range for stamps
            continue
        y0, x0, y1, x1 = region.bbox
        w, h = x1 - x0, y1 - y0
        aspect_ratio = w / float(h)
        density = region.area / (w * h)
        
        # Stamp characteristics: near-square, solid, dense
        if (0.7 < aspect_ratio < 1.5 and 
            region.solidity > 0.8 and  # More solid shape
            density > 0.5):  # Higher density due to filled areas
            # Set only the exact region pixels
            stamp_mask[blobs_labels == region.label] = 255
    
    return stamp_mask

def main():
    image_path = '/content/sample1.jpg'  # Replace with your input image path
    
    # Read the input image in grayscale
    img_gray = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
    
    if img_gray is None:
        print(f"Error: Could not load image at {image_path}")
        return
    
    # Detect text regions
    text_mask = detect_text_regions(img_gray)

    # Detect signatures (excluding text regions)
    signature_mask = detect_signatures(img_gray.copy(), text_mask)

    # Detect stamps
    stamp_mask = detect_stamps(img_gray.copy())

    # Read original image for highlighting purposes
    original_image = cv2.imread(image_path)

    # Create a version with text removed (white background)
    text_removed_image = original_image.copy()
    
    # Remove normal text by setting those pixels to white
    text_removed_image[text_mask > 0] = [255, 255, 255]

    # Highlight signatures in green (BGR: [0, 255, 0])
    highlighted_image = text_removed_image.copy()
    
    highlighted_image[signature_mask > 0] = [0, 255, 0]

    # Highlight stamps in red (BGR: [0, 0, 255])
    highlighted_image[stamp_mask > 0] = [0, 0, 255]

    # Save or display the highlighted output image
    output_path_highlighted = 'highlighted_output.jpg'
    cv2_imshow(highlighted_image)
    # If running locally, replace cv2_imshow with cv2.imshow
    # cv2.imshow('Highlighted Image', highlighted_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

SyntaxError: invalid syntax (557214332.py, line 5)